In [1]:
import spacy
from spacy import displacy
from spacy.util import filter_spans
import re
import pandas as pd
import pickle
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import Matcher 
from spacy.tokens import Span 
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
from tqdm import tqdm
from spacy.lang.en import English
%matplotlib inline

In [94]:
string = """Brown bears have long claws. 
The Brown bear has a brown fur. 
The Brown bear has a black nose.
The brown bear has a black nose.
The bear has a purple belly.
Their feet are orange.
The brown bear is similar to the black bear.
"""


In [118]:
string = """
Brown bears have very large and curved claws, those present on the forelimbs being longer than those on the hind limbs. They may reach 5 to 6 centimetres (2.0 to 2.4 in) and may measure 7 to 10 centimetres (2.8 to 3.9 in) along the curve.[69] They are generally dark with a light tip, with some forms having completely light claws.[62] Brown bear claws are longer and straighter than those of American black bears (Ursus americanus).[61] The claws are blunt, while those of a black bear are sharp. Due to their claw structure, in addition to their excessive weight, adult brown bears cannot typically climb trees as well as both species of black bear, although in rare cases adult female brown bears have been seen in trees.[70] The claws of a polar bear are also quite different, being notably shorter but broader with a strong curve and sharper point, presumably both as an aid to traveling over ice (sometimes nearly vertically) and procuring active prey.[23][71] The paws of the brown bear are quite large. The rear feet of adult bears have been found to typically measure 21 to 36 cm (8.3 to 14.2 in) long, while the forefeet tend to measure about 40% less in length. All four feet in average sized brown bears tend to be about 17.5 to 20 cm (6.9 to 7.9 in) in width. In large coastal or Kodiak bear males, the hindfoot may measure up to 40 cm (16 in) in length, 28.5 cm (11.2 in) in width, while outsized Kodiak bears having had confirmed measurements of up to 46 cm (18 in) along their rear foot.[72][73][74] Brown bears are the only extant bears with a hump at the top of their shoulder, which is made entirely of muscle, this feature having developed presumably for imparting more force in digging, which is habitual during foraging for most bears of the species and also used heavily in den construction prior to hibernation.[61][75] The brown bear's strength has been roughly estimated as 2.5 to 5 times that of a human.[76]
"""

In [119]:
def TextCleaner(text, splitComma=False, splitPointComma=False):
    
    """
    Split text into sentences and cleans them 
    from e.g. whitespaces and references.
    """
    
    # Split on commas
    if splitComma:
        text = text.replace(', ', '. ')
    if splitPointComma:
        text = text.replace('; ', '. ')
    text = nlp(text)
    # Init result
    result = []
    # Init regexes
    TextCleaner = [
            ' \(\d+.+?Close\n\t\n\)',
            '\[\d*\]',
            '\([^)]*\)'
    ]
    # Extract single senteces if large enough
    sentences = [str(sent) for sent in text.sents if len(sent) >= 3]
        
    for sentence in sentences:
        #print(sentence.lower())
        # Apply the regexes
        for Cleaner in TextCleaner:
            sentence = re.sub(Cleaner, '', sentence, flags=re.DOTALL)
        # Clean tabs and newlines
        sentence = sentence.replace('\n', '')
        sentence = sentence.replace('\t', '')
        # Add to result
        result.append(sentence.lower())
    
    return result

In [120]:
sentences_cleaned = TextCleaner(string)
sentences_cleaned

['brown bears have very large and curved claws, those present on the forelimbs being longer than those on the hind limbs.',
 'they may reach 5 to 6 centimetres  and may measure 7 to 10 centimetres  along the curve.',
 'they are generally dark with a light tip, with some forms having completely light claws.',
 'brown bear claws are longer and straighter than those of american black bears .',
 'the claws are blunt, while those of a black bear are sharp.',
 'due to their claw structure, in addition to their excessive weight, adult brown bears cannot typically climb trees as well as both species of black bear, although in rare cases adult female brown bears have been seen in trees.',
 'the claws of a polar bear are also quite different, being notably shorter but broader with a strong curve and sharper point, presumably both as an aid to traveling over ice  and procuring active prey.',
 'the paws of the brown bear are quite large.',
 'the rear feet of adult bears have been found to typicall

In [122]:
sentence = nlp(sentences_cleaned[0])
test = [i.root.head for i in sentence.noun_chunks]

In [133]:
[chunks.text for chunks in sentence.noun_chunks]

['brown bears',
 'very large and curved claws',
 'the forelimbs',
 'the hind limbs']

In [132]:
list(dict.fromkeys(test))

[have, on, on]

In [144]:
def NounChunkExtractor(sentence):
    
    """
    Extracts the noun chunks of a sentence.
    """
    
    # Nlp the sentence
    sentence = nlp(sentence)
    
    # Extract chunks
    chunks = [chunks.text for chunks in sentence.noun_chunks]
    # Extract relations
    relation = [roots.root.head for roots in sentence.noun_chunks]
    # Drop doubles
    relation = list(dict.fromkeys(relation))
    
    # Create triples
    RDF_1 = (chunks[0], relation[0], chunks[1])
    RDF_2 = (chunks[2], relation[1], chunks[3])

    """
    # Extract chunks
    for chunks in sentence.noun_chunks:
        #print(chunks)
        #print(chunks.root.head)
        ChunkList.append(chunks.text)
        ChunkList.append(chunks.root.head.text)
    """
        
    return RDF_1, RDF_2
    

In [149]:
NounChunkExtractor(sentences_cleaned[2])

(('they', are, 'a light tip'), ('some forms', with, 'completely light claws'))

In [150]:
displacy.render(nlp(sentences_cleaned[2]), style="dep")